In [6]:
from contextlib import redirect_stdout
import random
from functools import lru_cache

import os
import numpy as np
import qutip

from exputils.state.random import (
    _dm_check,
    _make_random_mixed_density_matrix,
    _make_random_pure_density_matrix,
)
from exputils.state.state_in_pauli_basis import state_in_pauli_basis


@lru_cache
def M(n):
    # defined in the our paper
    assert 1 <= n <= 5
    if n == 1:
        return np.array(
            [
                [1, 0, 0, 1],
                [0, 1, 1, 0],
                [0, 1j, -1j, 0],
                [1, 0, 0, -1],
            ]
        )
    else:
        return np.kron(M(n - 1), M(1))


# assert M_n^\dagger M_n = 2^n I
for n in range(1, 5):
    mat = M(n).conj().T @ M(n)
    assert np.all(abs(mat.imag) < 1e-10)
    assert np.all(abs(mat.real - (2**n) * np.eye(4**n)) < 1e-10)


# https://www.geeksforgeeks.org/moser-de-bruijn-sequence/
def _Moser_de_Bruijn_sequence(n):
    assert 0 <= n
    if n == 0:
        return 0
    if n == 1:
        return 1
    if n % 2 == 0:
        return 4 * _Moser_de_Bruijn_sequence(n // 2)
    else:
        return 4 * _Moser_de_Bruijn_sequence(n // 2) + 1


@lru_cache(maxsize=100)
def Moser_de_Bruijn_sequence(n):
    return [_Moser_de_Bruijn_sequence(i) for i in range(2**n)]


# https://en.wikipedia.org/wiki/Moser%E2%80%93de_Bruijn_sequence
def z_curve_ordering(dm: np.ndarray):
    assert dm.ndim == 2
    assert dm.shape[0] == dm.shape[1]
    n = dm.shape[0]
    assert bin(n).count("1") == 1
    n = int(np.log2(n))
    seq = Moser_de_Bruijn_sequence(n)
    idxs = np.zeros((2**n, 2**n), dtype=int)
    for i in range(2**n):
        for j in range(2**n):
            idxs[j, i] = seq[i] + 2 * seq[j]
    ret = np.zeros(dm.size, dtype=dm.dtype)
    for i in range(2**n):
        for j in range(2**n):
            ret[idxs[i, j]] = dm[i, j]
    return ret


with redirect_stdout(open(os.devnull, "w")):
    for n_qubit in range(1, 3 + 1):
        for trial in range(100):
            kets = [
                qutip.rand_ket_haar(2**n_qubit, seed=random.randint(0, 100000))
                for _ in range(2)
            ]
            dms = [qutip.ket2dm(ket).full() for ket in kets]
            _dm_check(dms[0])
            _dm_check(dms[1])
            pauli_vectors = [state_in_pauli_basis(dm, check=True) for dm in dms]

            stab_fidelity = abs((kets[0].dag() * kets[1])[0, 0]) ** 2
            tr = np.trace(dms[0] @ dms[1])
            dot = np.dot(pauli_vectors[0], pauli_vectors[1]) / (2**n_qubit)
            assert np.isclose(stab_fidelity, tr.real)
            assert np.isclose(stab_fidelity, dot)

    for n_qubit in range(1, 3 + 1):
        for trial in range(100):
            dm1 = _make_random_pure_density_matrix(
                n_qubit, seed=random.randint(0, 100000)
            )
            dm2 = _make_random_mixed_density_matrix(
                n_qubit, seed=random.randint(0, 100000)
            )
            _dm_check(dm1)
            _dm_check(dm2)
            pauli_vector_1 = state_in_pauli_basis(dm1, check=True)
            pauli_vector_2 = state_in_pauli_basis(dm2, check=True)
            assert np.allclose(
                (M(n_qubit) @ z_curve_ordering(dm1)).real,
                pauli_vector_1,
            )
            assert np.allclose(
                (M(n_qubit) @ z_curve_ordering(dm2)).real,
                pauli_vector_2,
            )

            dot = np.dot(pauli_vector_1, pauli_vector_2)

            Mdm_1 = M(n_qubit) @ z_curve_ordering(dm1)
            Mdm_2 = M(n_qubit) @ z_curve_ordering(dm2)
            assert Mdm_1.ndim == Mdm_2.ndim == 1
            Mdm_1_Mdm_2 = np.dot(Mdm_1, Mdm_2)
            assert np.isclose(Mdm_1_Mdm_2.imag, 0)
            assert np.isclose(Mdm_1_Mdm_2.real, dot)

            frobenius = 0
            for i in range(2**n_qubit):
                for j in range(2**n_qubit):
                    frobenius += dm1[i, j].conj() * dm2[i, j]
            assert np.isclose(frobenius.imag, 0)
            assert np.isclose((2**n_qubit) * frobenius.real, dot)

            trace = np.trace(dm1 @ dm2)
            assert np.isclose(trace.imag, 0)
            assert np.isclose((2**n_qubit) * trace.real, dot)

            dm1 = qutip.Qobj(dm1)
            dm2 = qutip.Qobj(dm2)

            assert np.isclose(qutip.fidelity(dm1, dm2) ** 2, trace.real, atol=1e-5)
            assert np.isclose(qutip.fidelity(dm1, dm2) ** 2, dot / (2**n_qubit))

print("OK!")

OK!


In "Simulation of quantum circuits by low-rank stabilizer decompositions"

(Sergey Bravyi, Dan Browne, Padraic Calpin, Earl Campbell, David Gosset, and Mark Howard)

It is said that $F(T) \approx 0.853$.

We confirm this.


In [9]:
from exputils.state.tensor import make_random_tensor_product_state
from exputils.actual_Amat import get_actual_Amat

n_qubit = 1
rho_vec = make_random_tensor_product_state("H", n_qubit, seed=0)
Amat = get_actual_Amat(n_qubit)
dots = rho_vec.T @ Amat
stabilizer_fidelity = np.max(dots) / (2**n_qubit)
print(stabilizer_fidelity)

0.8535533905932737
